<b><font color = "blue">Fake News Classifier </font></b><br>
<b>Dataset : <font color = "blue"> https://www.kaggle.com/c/fake-news/data# </font></b><br>

In [48]:
import pandas as pd

In [49]:
df = pd.read_csv('train.csv')

In [50]:
df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [51]:
## Get Independent Features 

x = df.drop('label',axis=1)

In [52]:
x.head(2)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...


In [53]:
df.shape

(20800, 5)

In [54]:
df = df.dropna()

In [55]:
df.shape

(18285, 5)

In [56]:
y = df['label']

In [57]:
y.shape

(18285,)

In [58]:
import tensorflow as tf
tf.__version__

'2.3.1'

In [59]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [60]:
## Vocabulay Size
voc_size = 5000

## One Hot Representation

In [61]:
message = df.copy()

In [62]:
message.reset_index(inplace=True)

In [63]:
## EDA 

import re
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords

ps = PorterStemmer()
corpus =[]

for i in range(0 , len(message)):
    review = re.sub('[^a-zA-Z]',' ',message['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [64]:
onehot_re = [one_hot(word,voc_size) for word in corpus]
onehot_re

[[142, 891, 36, 4238, 3660, 909, 4867, 883, 297, 4490],
 [3180, 2414, 2063, 1576, 2004, 4020, 2340],
 [3859, 2119, 1095, 4432],
 [679, 3679, 1030, 3236, 4023, 1404],
 [2788, 2004, 2765, 2983, 4404, 1550, 2004, 1136, 3550, 4870],
 [3018,
  1488,
  4530,
  1343,
  700,
  4017,
  4890,
  1398,
  3410,
  21,
  2785,
  2694,
  2681,
  2322,
  2340],
 [1310, 1367, 3701, 2152, 3638, 2294, 33, 1070, 2138, 2063, 2969],
 [1197, 1732, 432, 3532, 131, 1243, 4017, 3551, 2138, 2063, 2969],
 [313, 512, 2306, 2874, 1109, 3147, 984, 2493, 4017, 2693],
 [4865, 4758, 4144, 4739, 3520, 4092, 2029, 3617],
 [434, 1425, 3169, 2204, 2402, 3270, 3145, 1725, 669, 516, 1989],
 [3236, 1990, 3660, 3147, 4017, 131],
 [2694, 1012, 264, 598, 1410, 4190, 2349, 3300, 4756],
 [3749, 2018, 3367, 676, 4342, 4607, 3558, 2138, 2063, 2969],
 [672, 65, 4429, 3128, 3538, 2138, 2063, 2969],
 [208, 4280, 2964, 2905, 4432, 4480, 415, 2418, 4740, 2326],
 [3178, 4930, 2414],
 [4223, 4787, 4847, 946, 4017, 4310, 3889, 2340],
 [4670,

## Word Embedding Representation

In [65]:
sent_len = 20
embedded_doc = pad_sequences(onehot_re,padding='pre',maxlen=sent_len)
embedded_doc

array([[   0,    0,    0, ...,  883,  297, 4490],
       [   0,    0,    0, ..., 2004, 4020, 2340],
       [   0,    0,    0, ..., 2119, 1095, 4432],
       ...,
       [   0,    0,    0, ..., 2138, 2063, 2969],
       [   0,    0,    0, ..., 2860, 3195,  214],
       [   0,    0,    0, ..., 2038, 4803, 4174]])

In [101]:
## Creating model 
embedding_vec_feature = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vec_feature,input_length= sent_len))
model.add(LSTM(100)) ## 100 stands for 100 neuron in 1 LSTM
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy' ,optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [102]:
import numpy as np
x = np.array(embedded_doc)
y = np.array(y)
print(x.shape,y.shape)

(18285, 20) (18285,)


In [103]:
from sklearn.model_selection import train_test_split

## Splitting Data in 30-70 % ratio 

In [104]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=.33,random_state=0)

## Model fitting 

In [91]:
model.fit(train_x,train_y,validation_data=(test_x,test_y),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1050 - accuracy: 0.9691 - val_loss: 0.0905 - val_accuracy: 0.9705
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0426 - accuracy: 0.9881 - val_loss: 0.1070 - val_accuracy: 0.9657
Epoch 3/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0185 - accuracy: 0.9958 - val_loss: 0.1423 - val_accuracy: 0.9655
Epoch 4/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0081 - accuracy: 0.9982 - val_loss: 0.1617 - val_accuracy: 0.9650
Epoch 5/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0044 - accuracy: 0.9993 - val_loss: 0.2040 - val_accuracy: 0.9630
Epoch 6/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0023 - accuracy: 0.9997 - val_loss: 0.2266 - val_accuracy: 0.9629
Epoch 7/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0014 - accuracy: 0.9998 - val_loss: 0.2530 - val_accuracy: 0.9626

## Accuracy Score 

In [93]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [97]:
pred_y = model.predict_classes(test_x)

In [98]:
confusion_matrix(test_y,pred_y)

array([[3272,  117],
       [ 111, 2535]], dtype=int64)

In [99]:
accuracy_score(test_y,pred_y)

0.9622203811101906

## Hyperparameter - Adding Dropout

In [106]:
from tensorflow.keras.layers import Dropout


## Creating model 

embedding_vect_feature = 40
model_hyper = Sequential()
model_hyper.add(Embedding(voc_size,embedding_vect_feature,input_length = sent_len))
model_hyper.add(Dropout(0.3))
model_hyper.add(LSTM(100))
model_hyper.add(Dropout(0.3))
model_hyper.add(Dense(1,activation='sigmoid'))
model_hyper.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model_hyper.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [107]:
model_hyper.fit(x,y,validation_data=(test_x,test_y),epochs=10,batch_size=64)

Epoch 1/10
286/286 [==============================] - 10s 34ms/step - loss: 0.2979 - accuracy: 0.8603 - val_loss: 0.1621 - val_accuracy: 0.9427
Epoch 2/10
286/286 [==============================] - 9s 31ms/step - loss: 0.1483 - accuracy: 0.9390 - val_loss: 0.1050 - val_accuracy: 0.9662
Epoch 3/10
286/286 [==============================] - 9s 31ms/step - loss: 0.1112 - accuracy: 0.9580 - val_loss: 0.0703 - val_accuracy: 0.9766
Epoch 4/10
286/286 [==============================] - 9s 32ms/step - loss: 0.0834 - accuracy: 0.9707 - val_loss: 0.0461 - val_accuracy: 0.9882
Epoch 5/10
286/286 [==============================] - 9s 31ms/step - loss: 0.0639 - accuracy: 0.9767 - val_loss: 0.0326 - val_accuracy: 0.9924
Epoch 6/10
286/286 [==============================] - 9s 31ms/step - loss: 0.0493 - accuracy: 0.9826 - val_loss: 0.0210 - val_accuracy: 0.9957
Epoch 7/10
286/286 [==============================] - 9s 31ms/step - loss: 0.0389 - accuracy: 0.9879 - val_loss: 0.0159 - val_accuracy: 0.996

## Accuracy Score By Hyperparameter

In [108]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [114]:
pred_y_hyper = model_hyper.predict_classes(test_x)

In [115]:
confusion_matrix(test_y,pred_y_hyper)

array([[3369,   20],
       [   1, 2645]], dtype=int64)

In [116]:
accuracy_score(test_y,pred_y_hyper)

0.9965202982601491